In [1]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objects as go

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
data = yf.download(["PINS"],start="2023-01-01", end="2025-05-10")
data = data[['Close']]
data['Returns'] = data['Close'].pct_change() 
short = 12
long  = 26
data['EMA_short']= data['Close'].ewm(span=short,adjust=False).mean()
data['EMA_long']= data['Close'].ewm(span=long,adjust=False).mean()
data['Signal'] = 0
data.loc[data['EMA_short'] > data['EMA_long'], 'Signal'] = 1
data.loc[data['EMA_short'] < data['EMA_long'], 'Signal'] = -1
data['Strategy'] = data['Returns'] *(data['EMA_short']>data['EMA_long']) - data['Returns']*(data['EMA_short']<data['EMA_long'])

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
strategy_cumulative = np.cumprod(1+data['Strategy'])
buy_and_hold = np.cumprod(1+data['Returns'])
returns_strategy = np.prod(1+data['Strategy']) ** (252/len(data['Strategy'])) -1 
returns_buy_n_hold =np.prod(1+data['Returns']) ** (252/len(data['Strategy'])) -1 

In [4]:
returns_strategy_pct = returns_strategy * 100  # 67.86%
returns_buy_n_hold_pct = returns_buy_n_hold * 100  # 10.99%
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=strategy_cumulative.index,
        y=strategy_cumulative,
        name=f'Exponential Moving Average Strategy ({returns_strategy_pct:.1f}%)',
        line=dict(color='royalblue', width=2),
        hovertemplate="<b>%{x|%Y-%m-%d}</b><br>Return: %{y:.2f}%<extra></extra>",
    )
)
fig.add_trace(
    go.Scatter(
        x=buy_and_hold.index,
        y=buy_and_hold,
        name=f'Buy & Hold ({returns_buy_n_hold_pct:.1f}%)',
        line=dict(color='firebrick', width=2, dash='dot'),
        hovertemplate="<b>%{x|%Y-%m-%d}</b><br>Return: %{y:.2f}%<extra></extra>",
    )
)

fig.add_annotation(
    xref='paper', yref='paper',
    x=0.95, y=0.95,
    text=f'<b>Annual Returns:</b><br>'
         f'EMA: <span style="color:royalblue">{returns_strategy_pct:.1f}%</span><br>'
         f'B&H: <span style="color:firebrick">{returns_buy_n_hold_pct:.1f}%</span>',
    showarrow=False,
    align='left',
    bordercolor='lightgray',
    borderwidth=1,
    bgcolor='white',
    font=dict(size=12)
)

fig.update_layout(
    title='<b>EMA Strategy vs Buy & Hold on Pinterest, Inc. (PINS)</b>',
    xaxis_title='Date',
    yaxis_title='Cumulative Returns',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
    ),
    template='plotly_white',
    margin=dict(l=50, r=50, t=80, b=50),
    height=600,
    width=1000,
)

fig.show()

In [39]:
!ls